In [59]:
import tensorflow as tf
import os
import pandas as pd
from tqdm import tqdm
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [60]:
data_root = "/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data"

In [65]:
data = pd.read_csv(os.path.join(data_root,'train-dataset.csv'))
# # test_data = pd.read_csv(os.path.join(data_root,'test-dataset.csv'))
# # all_data = pd.concat([train_data,test_data])
# all_data.to_csv("/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data/all_data.csv")

In [62]:
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,LabelEncoder

In [66]:
#object特征处理：suv
enc = LabelEncoder()
data['suv']  = enc.fit_transform(data.suv)

In [67]:
#时间特征处理
data['date'] = pd.to_datetime(data['logTs'], unit='ms') 
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

In [69]:
data.to_csv("/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data/data.csv")

In [71]:
##data
data_root = "/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data"
data_ds = tf.data.experimental.make_csv_dataset(
            os.path.join(data_root, "data.csv"),
            batch_size=64,
            label_name='label',
            # na_value="0",
            num_epochs=1,
            shuffle=True,
            ignore_errors=True)
#验证集
val_ds = data_ds.take(1000)
#训练集
train_ds = data_ds.skip(1000)

# #测试集
# test_ds = tf.data.experimental.make_csv_dataset(
#             os.path.join(data_root, "test-dataset.csv"),
#             batch_size=64,
#             # label_name='label',
#             # na_value="0",
#             num_epochs=1,
#             shuffle=True,
#             ignore_errors=True)

In [72]:
##config
EMBEDDING_SIZE = 10
HIDDEN_SIZE = 128

# 1.制作inputs

In [73]:
inputs = {
    'suv' : tf.keras.layers.Input(name='suv', shape=(), dtype='int32'),
    'operator' : tf.keras.layers.Input(name='operator', shape=(), dtype='int32'),
    'browserType' : tf.keras.layers.Input(name='browserType', shape=(), dtype='int32'),
    'deviceType' : tf.keras.layers.Input(name='deviceType', shape=(), dtype='int32'),
    'osType' : tf.keras.layers.Input(name='osType', shape=(), dtype='int32'),
    'province' : tf.keras.layers.Input(name='province', shape=(), dtype='int32'),
    'itemId' : tf.keras.layers.Input(name='itemId', shape=(), dtype='int32'),
    'city' : tf.keras.layers.Input(name='city', shape=(), dtype='int32'),
}

# 2.制作feature_columns

In [74]:
feature_columns = {}
feature_columns['suv'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('suv',700000,tf.int64),EMBEDDING_SIZE)
feature_columns['itemId'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('itemId',6000,tf.int64),EMBEDDING_SIZE)
feature_columns['operator'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('operator',900,tf.int64),EMBEDDING_SIZE)
feature_columns['browserType'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('browserType',60,tf.int64) ,EMBEDDING_SIZE)
feature_columns['deviceType'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('deviceType',10,tf.int64) ,EMBEDDING_SIZE)
feature_columns['osType'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('osType',20,tf.int64) ,EMBEDDING_SIZE)
feature_columns['province'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('province',80,tf.int64) ,EMBEDDING_SIZE)
feature_columns['city'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_hash_bucket('city',800,tf.int64) ,EMBEDDING_SIZE)

In [75]:
user_col = [feature_columns['suv']]
user_features = tf.keras.layers.DenseFeatures(user_col)(inputs)
user_features

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_features_6')>

In [76]:
context_col = [feature_columns['operator'], 
               feature_columns['browserType'],
               feature_columns[ 'deviceType'], 
               feature_columns['osType'],
               feature_columns['province'],
               feature_columns['city']]
context_features = tf.keras.layers.DenseFeatures(context_col)(inputs)
context_features

<KerasTensor: shape=(None, 60) dtype=float32 (created by layer 'dense_features_7')>

In [77]:
item_col = [feature_columns['itemId']]
item_features = tf.keras.layers.DenseFeatures(item_col)(inputs)
item_features

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_features_8')>

In [78]:
deep_features = tf.concat([user_features,item_features,context_features],axis=-1)
deep_features

<KerasTensor: shape=(None, 80) dtype=float32 (created by layer 'tf.concat_14')>

In [79]:
wide_features = tf.concat([user_features,item_features,context_features],axis=-1)
wide_features

<KerasTensor: shape=(None, 80) dtype=float32 (created by layer 'tf.concat_15')>

In [80]:
deep_outputs = tf.keras.layers.Dense(1024,activation='relu')(deep_features)
deep_outputs = tf.keras.layers.Dense(512,activation='relu')(deep_outputs)
deep_outputs = tf.keras.layers.Dense(256,activation='relu')(deep_outputs)

In [81]:
wide_outputs = wide_features

In [82]:
outputs = tf.concat([wide_outputs,deep_outputs],axis=-1)
outputs

<KerasTensor: shape=(None, 336) dtype=float32 (created by layer 'tf.concat_16')>

In [83]:
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(outputs)

In [84]:
model = tf.keras.Model(inputs, outputs)

In [85]:
METRICS = [
      # tf.keras.metrics.TruePositives(name='tp'),
      # tf.keras.metrics.FalsePositives(name='fp'),
      # tf.keras.metrics.TrueNegatives(name='tn'),
      # tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      # tf.keras.metrics.Precision(name='precision'),
      # tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [86]:
model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=METRICS)

#train
history = model.fit(train_ds, epochs=1)

/root/miniconda3/lib/python3.8/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['', 'sampleId', 'pvId', 'userSeq', 'logTs', 'date', 'year', 'month', 'day'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
2023-01-28 14:19:11.055460: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


65154/65154 [==============================] - 872s 13ms/step - loss: 0.4853 - accuracy: 0.8074 - auc: 0.5613


In [87]:
val_loss, val_accuracy,val_auc = model.evaluate(val_ds)

1000/1000 [==============================] - 12s 11ms/step - loss: 0.5266 - accuracy: 0.7877 - auc: 0.5080
